In [1]:
# Creating the postcode centroid file
# I'm importing argparse so I can use it to sync devices from cloud APIs, ingest the postcode centroids, and fill any missing historicla data if required.
# Importing logging to make debugging easier as it'll tell me where the issue is, for example which API link may be causing the issue.
import os
import sys
import time
import json
import argparse
import logging

# Importing requests so that my postcode centroid URL can be fetched and used.
# SQLAlchemy helps connect me to TimeScaleDB for the dashboard and automatically reconnects to Render.
import requests
from sqlalchemy import create_engine, text

In [ ]:
from IPython.core.interactiveshell import page
postcode_centroid_URL = "https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/ONSPD_LATEST_UK/FeatureServer/1"

# Through normalisation, I will ensure that all the postcodes are in the correct format
def normalisation(postalcode):
  p = postalcode.strip().upper().replace("","")
  if len(p) > 3:
    p = p[:-3]+""+ p[-3:]
  return p

# Since I know the max record count is 2000 and features of the URL from the directory
# Creating a function so that I can later call it to create pages of london centroid postcodes with the chosen features.
def query_page(postcode_centroid_URL,where,offset,page_size,session,timeout = 120, retries = 4):
  parameters = {
      "where": where,
      "resultOffset": offset,
      "outFields": "PCDS,LONG,LAT,OBJECTID,LAD25CD",
      "f": "json",
      "resultRecordCount": page_size,
      "orderByFields":"OBJECTID"
      }
  response = session.get(postcode_centroid_URL + "/query",params = parameters)
  data = response.json()
# Getting the attributes dictionary
  return [f["attributes"] for f in data.get("features",[])]

# I am now calling all the London postcode centroids into pages through paginiation and filtering out the wanted features

offset = 0
session = requests.Session()
all_london_rows = []

while True:
  page = query_page(
      postcode_centroid_URL,
      where = "LAD25CD LIKE 'E090%'",
      offset = offset,
      page_size = 2000,
      session = session
  )
  if not page:
    break

  # Making sure the loop continues until there are no more postcodes

  all_london_rows.extend(page)
  offset += len(page)
